In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'GPU.CPU: {device}')

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

GPU.CPU: cpu


In [2]:
batch_size = 64
lr = 0.001
num_epochs = 20
num_classes = 102

# 1. 전처리

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
])
val_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
])

# 2. 데이터 불러오기

In [4]:
train_dataset = datasets.FashionMNIST(root='data', transform=train_transform, download=True)
val_dataset = datasets.FashionMNIST(root='data', transform=val_transform, download=True)
test_dataset = datasets.FashionMNIST(root='data', transform=val_transform, download=True)

100.0%
100.0%
100.0%
100.0%


In [5]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [6]:
print(f'훈련 데이터: {len(train_loader)}')
print(f'검증 데이터: {len(val_loader)}')
print(f'테스트 데이터: {len(test_loader)}')

훈련 데이터: 938
검증 데이터: 938
테스트 데이터: 938


In [7]:
data_iter = iter(train_loader)
sample_batch = next(data_iter)
sample_image, sample_label = sample_batch
sample_image.shape

torch.Size([64, 1, 32, 32])

# 3. 모델 설계

In [ ]:
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), # 첫번째 매개변수(gray||RGB)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(64*7*7, 512)
        self.drop = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.drop(x)
        x = self.fc2(x)
        return x

# 4. 학습

In [16]:
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # 순전파
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 역전파 및 최적화
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

model = FashionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model, train_loader, criterion, optimizer, num_epochs)

Epoch [1/20], Loss: 0.5055
Epoch [2/20], Loss: 0.3074
Epoch [3/20], Loss: 0.2634
Epoch [4/20], Loss: 0.2376
Epoch [5/20], Loss: 0.2190
Epoch [6/20], Loss: 0.2057
Epoch [7/20], Loss: 0.1916
Epoch [8/20], Loss: 0.1818
Epoch [9/20], Loss: 0.1677
Epoch [10/20], Loss: 0.1584
Epoch [11/20], Loss: 0.1490
Epoch [12/20], Loss: 0.1451
Epoch [13/20], Loss: 0.1347
Epoch [14/20], Loss: 0.1285
Epoch [15/20], Loss: 0.1242
Epoch [16/20], Loss: 0.1167
Epoch [17/20], Loss: 0.1118
Epoch [18/20], Loss: 0.1073
Epoch [19/20], Loss: 0.1026
Epoch [20/20], Loss: 0.0994


# 5. 평가

In [17]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
evaluate_model(model, test_loader)

Test Accuracy: 97.03%
